In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [25]:
base_path = './data'

path_player_away_train = f'{base_path}/Train_Data/train_away_player_statistics_df.csv'
path_player_home_train = f'{base_path}/Train_Data/train_home_player_statistics_df.csv'
path_team_away_train = f'{base_path}/Train_Data/train_away_team_statistics_df.csv'
path_team_home_train = f'{base_path}/Train_Data/train_home_team_statistics_df.csv'
path_target = f'{base_path}/Y_train_1rknArQ.csv'
path_target_sup = f'{base_path}/benchmark_and_extras/Y_train_supp.csv'

In [26]:
train_target = pd.read_csv(path_target, sep=',')

In [27]:
conditions = [
    train_target['HOME_WINS'] == 1, 
    train_target['DRAW'] == 1      
]

values = [1, 0]

train_target['RESULTAT'] = np.select(conditions, values, default=2)

train_target.head()

,ID,HOME_WINS,DRAW,AWAY_WINS,RESULTAT
0,0,0,0,1,2
1,1,0,1,0,0
2,2,0,0,1,2
3,3,1,0,0,1
4,4,0,1,0,0


In [28]:
train_team = pd.read_csv('train_team.csv', sep = ',' )
train_team.head()

,ID,HOME_LEAGUE,HOME_TEAM_NAME,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,...,AWAY_TEAM_YELLOWCARDS_5_last_match_std,AWAY_TEAM_REDCARDS_5_last_match_std,AWAY_TEAM_OFFSIDES_5_last_match_std,AWAY_TEAM_ATTACKS_5_last_match_std,AWAY_TEAM_PENALTIES_5_last_match_std,AWAY_TEAM_SUBSTITUTIONS_5_last_match_std,AWAY_TEAM_BALL_SAFE_5_last_match_std,AWAY_TEAM_DANGEROUS_ATTACKS_5_last_match_std,AWAY_TEAM_INJURIES_5_last_match_std,AWAY_TEAM_GOALS_5_last_match_std
0,0,Ligue 1,Toulouse,3.0,2.0,5.0,2.0,1.0,2.0,2.0,...,5.0,5.0,4.0,0.0,6.0,8.0,4.0,3.0,2.0,3.0
1,1,Ligue 2,Brest,6.0,8.0,3.0,6.0,5.0,8.0,7.0,...,0.0,0.0,3.0,1.0,8.0,4.0,10.0,0.0,5.0,3.0
2,2,Serie A,Sampdoria,4.0,2.0,5.0,2.0,8.0,1.0,1.0,...,6.0,10.0,4.0,4.0,0.0,8.0,3.0,0.0,9.0,6.0
3,3,League One,Coventry City,7.0,5.0,5.0,6.0,6.0,9.0,9.0,...,0.0,0.0,1.0,2.0,0.0,5.0,6.0,3.0,NaN,2.0
4,4,Premier League,Wolverhampton Wanderers,3.0,3.0,2.0,3.0,4.0,4.0,3.0,...,1.0,0.0,4.0,4.0,9.0,4.0,1.0,4.0,6.0,5.0


In [29]:
X_train, X_test, y_train, y_test = train_test_split(train_team, train_target['RESULTAT'], test_size=0.2, random_state=42)

In [30]:
print(f"Nombre de données d'entrainement: {X_train.shape[0]}")
print(f"Nombre de données de test: {X_test.shape[0]}")

Nombre de données d'entrainement: 9842
Nombre de données de test: 2461


In [ ]:
rf_reg_params = {'n_estimators': range(5, 100, 20)}

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

rf_reg = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=rf_reg_params,
    n_jobs=-1,
    cv=kfold,
    scoring='neg_mean_squared_error' 
)

rf_reg.fit(X_train, y_train)

cv_results = rf_reg.cv_results_['mean_test_score']
best_params = rf_reg.best_params_
best_rf_reg = rf_reg.best_estimator_

test_mse = mean_squared_error(y_test, best_rf_reg.predict(X_test))
print("Best parameters for RandomForestRegressor:", best_params)
print("Test MSE for RandomForestRegressor:", test_mse)

rf_clf = RandomForestClassifier(n_estimators=20, max_features=4, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy for RandomForestClassifier:", test_accuracy)

ValueError: 
All the 25 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/leonardo/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/leonardo/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/leonardo/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py", line 363, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/leonardo/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 650, in